In [1]:
import pandas as pd
import numpy as np

roleTest=pd.read_excel('./roleTest.xlsx',sheet_name='roleDamage')
roleCompare=pd.read_excel('./roleTest.xlsx',sheet_name='roleCompare')

因为我只有刃和希尔两个 生命/攻击，对单/对群 活动角色，这里的数值模拟暂时按照我对这两个角色的理解作为标准.  
数值模拟使用我自己搭建的简单伤害模拟器。链接：http://1.13.254.14/SR/damageSimulator/dist 。

# 基本思路

>## 玩法思路
>>1. 通过调整弹药数量，该角色可在生命值加成的群体输出/爆发输出的低速角色和攻击力加成的持续稳定单体输出的高速角色之间调整。通过大招全弹发射保证两种玩法的对单输出下限。

>>2. 对于高速单体流，角色主要输出手段为保持1、2弹状态下强化普攻的持续单体输出，充能完毕后可以打一波小爆发。

>>3. 对于低速群伤流，角色主要输出手段为保持3、4弹状态的群体炮击。在充能结束后，可对boss敌人造成一次毁灭性打击。

>>4. 对于混合培养的玩家，角色将在战斗中控制弹药数量，满足不同战况的需求。相应的，由于灵活性增加，输出能力会低于上面两者。

>## 数值设计思路
>### 前提
>>1. 星穹铁道期望伤害为 倍率 *（1+生命/攻击加成）*（1+暴击率*暴击伤害）*（1+增伤）。
>>在现有角色培养体系下，最后角色+专武每个板块的加成约为 150%±20% ，伤害总系数约为 2.5的三次方，以保证输出能力与吃buff能力的均衡。
>>因此自定义角色在不同输出培养体系下，不应留出过大的增伤空白区间，以预防吃buff太厉害。

>### 角色约束

>>1. 对于高速普攻流，期望角色始终从攻击/双爆加成中获得最大收益；对于低速炮击流，期望角色始终从生命加成中获得最大收益，以防止混合流获取超出预期的收益，降低另外两种流派的特色。

>>2. 普攻流始终从更低的炮弹装填量中获得收益；炮击流始终从更高的炮弹装填量中获得收益。

>>3. 由于炮弹流速度、循环更慢，希望其大招爆发更高作为补偿。

>>4. 限制角色最高对单、对群输出能力，防止角色超模

>### 结果

>>1. 角色共有3种输出方式：普攻、炮击与强化普攻，其中强化普攻为角色爆发伤害来源，输出公式为 炮弹数量*（普攻+炮击）。而星穹铁道中1点属性=1点攻击力=2点生命值，因此设计每单位普攻、炮击输出倍率为50%、100%，由此一点属性在基础倍率上的加成是无差异的。
>>2. 以2弹状态作为角色均衡状态，规定角色2弹时生命值刚好为攻击力的2倍，增加弹药会降低基础攻击力，提高基础生命值，因此弹药数大于2时，生命值比例高于攻击力比例，提高生命值的收益始终最大，反之提高攻击力。
>>3. 由于无视防御的炮击是一件很有特点的事情，基于炮弹数量直接造成k倍伤害也很有代入感，最终将炮击设计为不受防御影响不能暴击的伤害，并通过弹药数量动态调控增伤能力与暴伤能力，形成两种特色分明的输出方式。角色一弹的强化攻击倍率为2攻击+1生命，四弹的攻击倍率为5攻击+2生命，在生命与攻击加成分离的条件下倍率略高，但附带效果更少（没有再现、扩散等收益）。
>>4. 对于普攻流角色，在四弹爆发的情况下，其会失去攻击力和暴击能力的加成，由此突出炮击流的爆发能力。 
>>5. 以85级敌人为例，其伤害系数为0.439，无视防御的收益为1.2左右，以主C 70%暴击率 170%暴击伤害来看，暴击的期望收益约为1.2左右，平衡了输出能力。但是该角色可以将多余的词条用于基础面板提升，因此角色单人输出能力会更强，但由于角色已经有了很高的面板加成，吃buff能力下降，配合辅助的条件下输出能力更弱。

# 数值模拟

>## 面板模拟
>>数据模拟中，角色带专武装配物理、繁星套，全堆生命值时4弹状态下拥有最强对群和爆发，全堆攻击时1弹、2弹状态下拥有最强持续输出能力。下面模拟其面板。

>>攻击1弹

<img src='./assets/diy1A.png' width='60%' />

>>攻击2弹

<img src='./assets/diy2A.png' width='60%' />

>>生命4弹

<img src='./assets/diy4H.png' width='60%' />

>>混合2弹

<img src='./assets/diy2M.png' width='60%' />


# 伤害模拟

>## 角色伤害模拟

>>下图伤害模拟以我方80级敌方85级带盾0.439补正的状态为例
>>数字后缀代表剩余弹数，字母后缀A代表攻击力配装，H代表生命值配装，M代表混合配装

In [2]:
roleTest

,role,singleAttack,singlePao,allin,enemy3Pao,enemy5Pao
0,diy0A,11376,0,11376.00,0,0
1,diy1A,10147,4157,25490.25,12471,20785
2,diy2A,8697,4981,38543.50,14943,24905
3,diy3A,7309,5875,51267.25,17625,29375
4,diy4A,6224,7083,66535.00,21249,35415
5,diy0H,4918,6722,11640.00,20166,33610
6,diy1H,4295,8352,19030.00,25056,41760
7,diy2H,3709,10139,36474.50,30417,50695
8,diy3H,1371,12083,50795.25,36249,60415
9,diy4H,2782,14691,87365.00,44073,73455


>## 角色伤害对比

>>下图为DIY角色与刃、希尔直接伤害、带辅助伤害对比。

>>其中base代表希儿普攻，刃强化普攻，DIY角色普攻，constant代表希儿战技，刃追加攻击，DIY角色炮击,

>>immediate代表希儿大招，刃大招，DIY角色强化普攻。>>withYB代表带上银狼、布洛尼亚双辅助。

In [3]:
roleCompare

,role,base,constant,immediate
0,xier,11460,25213,57087.00
1,xierWithTingyun,16283,35824,82223.00
2,xierWithYinlang,18209,40060,90705.00
3,xierWithBuluo,21589,47497,105532.00
4,xierWithYB,34303,75467,167679.00
5,ren,21571,23729,39567.00
6,renWithTingyun,27124,29837,49159.00
7,renWithYinlang,33459,36805,61371.00
8,renWithBuluo,35338,38872,63969.00
9,renWithYB,54811,60292,99219.00


>>从上表看出单人状态下，对单方面爆发方面，希儿一回合大招爆发5.5w+战技2.5w=8w，diy角色生命值4弹状态下一回合最高爆发8w。

>>持续输出方面，希尔与刃能单体输出25000左右的伤害，而diy角色1弹、2弹攻击力状态下能稳定输出2w到3w的单体伤害。

>>对群方面，刃的追加攻击能造成2w左右伤害，普攻拥有1w左右的扩散伤害，diy角色生命值4弹状态下炮弹可造成每人1.5w的群体伤害。

>>不出所料的diy角色吃buff能力相对更弱，配合辅助的增幅不如另外两位，这里也没有考虑希儿的再现和刃的回复能力。但diy角色输出稳定，输出方式灵活，也有机会打出双全弹的高爆发。

>>综合来看，diy角色强度符合预期。


